In [20]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer

## Data processing ##

In [151]:
hands = pd.read_csv('./hands_data.csv')
hands.drop(columns = 'Unnamed: 0')

,video,letter,handedness,x0,x1,x2,x3,x4,x5,x6,...,y18,y19,y20,bbox_x1,bbox_y1,bbox_x2,bbox_y2,iou0,iou1,iou2
0,vid3,A,Right,0.362896,0.418465,0.486683,0.546821,0.595302,0.463339,0.444256,...,0.321185,0.344583,0.349709,0.320133,0.297793,0.595302,0.437581,NaN,NaN,NaN
1,vid3,A,Right,0.358468,0.413572,0.485543,0.546373,0.594428,0.468047,0.448165,...,0.319683,0.342699,0.347909,0.322879,0.297573,0.594428,0.433684,0.957844,NaN,NaN
2,vid3,A,Right,0.360684,0.413036,0.483942,0.544283,0.593048,0.463905,0.446882,...,0.319794,0.342711,0.347140,0.322172,0.298325,0.593048,0.435321,0.975200,0.957844,NaN
3,vid3,A,Right,0.349367,0.405749,0.475828,0.534505,0.581794,0.451813,0.425878,...,0.314091,0.339722,0.347585,0.303982,0.288935,0.581794,0.428068,0.800520,0.975200,0.957844
4,vid3,A,Right,0.354493,0.408133,0.479159,0.538913,0.586343,0.455486,0.434271,...,0.322346,0.346321,0.351305,0.311088,0.297836,0.586343,0.437049,0.845080,0.800520,0.975200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2354,vid4,YA,Left,0.664645,0.587001,0.568673,0.646658,0.749332,0.519975,0.545874,...,0.570490,0.614499,0.652892,0.519975,0.325261,0.799866,0.833102,0.363197,0.782826,0.852497
2355,vid4,YA,Left,0.560196,0.458429,0.417879,0.485016,0.596180,0.428843,0.431605,...,0.565826,0.610831,0.650451,0.417879,0.346134,0.669148,0.806221,0.362918,0.363197,0.782826
2356,vid4,YA,Left,0.567616,0.450986,0.411073,0.477259,0.590838,0.432185,0.441896,...,0.545943,0.596377,0.643310,0.411073,0.330781,0.673923,0.808258,0.921122,0.362918,0.363197
2357,vid4,YA,Left,0.590661,0.483118,0.446346,0.517119,0.633998,0.462151,0.477644,...,0.570604,0.622106,0.665234,0.446346,0.336697,0.707881,0.819620,0.742729,0.921122,0.362918


In [172]:
hands.loc[hands.x4 < 0]

,Unnamed: 0,video,letter,handedness,x0,x1,x2,x3,x4,x5,...,y18,y19,y20,bbox_x1,bbox_y1,bbox_x2,bbox_y2,iou0,iou1,iou2
734,734,vid1,Ik,Left,0.156119,0.093719,0.053262,0.028271,-0.002452,0.093813,...,0.381901,0.359757,0.338380,-0.002452,0.313393,0.268897,0.533898,0.000000,0.077774,0.182312
747,747,vid4,Ik,Left,0.297256,0.173787,0.080157,-0.009555,-0.082626,0.216251,...,0.656086,0.608321,0.557665,-0.118281,0.525535,0.663790,0.901383,0.037729,0.306434,0.076269
748,748,vid4,Ik,Left,0.245022,0.135383,0.061990,-0.013852,-0.095903,0.205659,...,0.647367,0.601523,0.553740,-0.121703,0.483689,0.657031,0.911789,0.867475,0.037729,0.306434
2177,2177,vid4,Yrm,Left,0.425461,0.276353,0.131849,0.002720,-0.105862,0.308427,...,0.745859,0.795077,0.817767,-0.105862,0.410440,0.578166,0.924163,0.528466,0.365994,NaN


In [171]:
hands.describe()

,Unnamed: 0,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,y18,y19,y20,bbox_x1,bbox_y1,bbox_x2,bbox_y2,iou0,iou1,iou2
count,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,...,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2239.000000,2119.000000,1999.000000
mean,1179.000000,0.491962,0.480395,0.476128,0.479267,0.483697,0.477050,0.477822,0.476106,0.474847,...,0.465471,0.468672,0.467610,0.330134,0.328370,0.650470,0.611967,0.539560,0.535026,0.530037
std,681.128965,0.112967,0.107115,0.133805,0.149596,0.166161,0.106558,0.118616,0.131305,0.145254,...,0.114410,0.123132,0.131845,0.108110,0.092815,0.112618,0.127942,0.270321,0.267740,0.268405
min,0.000000,0.084367,0.093719,0.053262,-0.013852,-0.105862,-0.014298,-0.005807,-0.006934,-0.121703,...,0.103803,0.104387,0.080869,-0.121703,-0.035481,0.268897,0.287404,0.000000,0.000000,0.000000
25%,589.500000,0.419438,0.408134,0.375111,0.365217,0.367889,0.403265,0.399570,0.388666,0.379681,...,0.383587,0.382777,0.377088,0.260774,0.273680,0.573007,0.519752,0.357718,0.357718,0.352604
50%,1179.000000,0.488543,0.473495,0.460931,0.477259,0.486890,0.471849,0.471457,0.472337,0.470498,...,0.464664,0.460047,0.458344,0.329921,0.322237,0.645550,0.600240,0.578939,0.572532,0.568044
75%,1768.500000,0.566695,0.545595,0.572587,0.579389,0.601279,0.545289,0.553521,0.563732,0.571797,...,0.536922,0.550796,0.554398,0.393986,0.379493,0.720002,0.700392,0.755326,0.750631,0.747105
max,2358.000000,0.895753,0.993937,1.066815,1.112876,1.135005,0.994078,0.994166,1.072315,1.131592,...,1.051432,1.034735,1.006614,0.822089,0.815160,1.135005,1.090507,0.990327,0.990327,0.990327


In [152]:
xs_cols = [f'x{i}' for i in range(21)]
ys_cols = [f'y{i}' for i in range(21)]
landarks_cols = xs_cols + ys_cols

In [153]:
def mirror_transformer(X):
    """
    Apply (1 - val) for all rows that have class 'A' in feature1.
    This function assumes the input X is a pandas DataFrame.
    """
    X = X.copy()
    print (X.columns)
    
    # X.loc[X['handedness'] == 'Left', 'feature1'] = 1 - X.loc[mask, 'feature1']
    X.loc[X['handedness'] == 'Left', xs_cols] = X.loc[X['handedness'] == 'Left', xs_cols].apply(lambda x: 1 - x)
    return X

# FunctionTransformer requires the function to accept and return a numpy array
def mirror_hand_transformer_np(X):
    X_df = pd.DataFrame(X, columns = ['handedness'] + xs_cols)  # Convert to DataFrame
    X_transformed = mirror_transformer(X_df)
    return X_transformed.values  # Convert back to numpy array

In [ ]:
def scale_hand_transformer(X):
    """
        sets hand landmarks to start from 0 and end at 1
    """
    X = X.copy()
    
    # X.loc[X['handedness'] == 'Left', 'feature1'] = 1 - X.loc[mask, 'feature1']
    X.loc[X['handedness'] == 'Left', xs_cols] = X.loc[X['handedness'] == 'Left', xs_cols].apply(lambda x: 1 - x)
    return X

# FunctionTransformer requires the function to accept and return a numpy array
def scale_hand_transformer_np(X):
    X_df = pd.DataFrame(X, columns = ['handedness'] + xs_cols)  # Convert to DataFrame
    X_transformed = mirror_transformer(X_df)
    return X_transformed.values  # Convert back to numpy array

In [165]:
scaled_landmarks = ColumnTransformer(transformers = [
	# ("landmarks_width", MinMaxScaler(), xs_cols),
	# ("landmarks_height", MinMaxScaler(), ys_cols),
    ("handedness", FunctionTransformer(mirror_hand_transformer_np), xs_cols)
], remainder = "passthrough")

# scaled_landmarks.set_output(transform = 'pandas')

In [166]:
scaled_hands = scaled_landmarks.fit_transform(hands)

Index(['handedness', 'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8',
       'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18',
       'x19', 'x20'],
      dtype='object')


In [167]:
pd.DataFrame(scaled_hands)

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,NaN,0.362896,0.418465,0.486683,0.546821,0.595302,0.463339,0.444256,0.436085,0.447892,...,0.321185,0.344583,0.349709,0.320133,0.297793,0.595302,0.437581,NaN,NaN,NaN
1,NaN,0.358468,0.413572,0.485543,0.546373,0.594428,0.468047,0.448165,0.437576,0.44914,...,0.319683,0.342699,0.347909,0.322879,0.297573,0.594428,0.433684,0.957844,NaN,NaN
2,NaN,0.360684,0.413036,0.483942,0.544283,0.593048,0.463905,0.446882,0.436377,0.447657,...,0.319794,0.342711,0.34714,0.322172,0.298325,0.593048,0.435321,0.9752,0.957844,NaN
3,NaN,0.349367,0.405749,0.475828,0.534505,0.581794,0.451813,0.425878,0.418438,0.43408,...,0.314091,0.339722,0.347585,0.303982,0.288935,0.581794,0.428068,0.80052,0.9752,0.957844
4,NaN,0.354493,0.408133,0.479159,0.538913,0.586343,0.455486,0.434271,0.424735,0.436329,...,0.322346,0.346321,0.351305,0.311088,0.297836,0.586343,0.437049,0.84508,0.80052,0.9752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2354,NaN,0.664645,0.587001,0.568673,0.646658,0.749332,0.519975,0.545874,0.573711,0.597614,...,0.57049,0.614499,0.652892,0.519975,0.325261,0.799866,0.833102,0.363197,0.782826,0.852497
2355,NaN,0.560196,0.458429,0.417879,0.485016,0.59618,0.428843,0.431605,0.44451,0.460421,...,0.565826,0.610831,0.650451,0.417879,0.346134,0.669148,0.806221,0.362918,0.363197,0.782826
2356,NaN,0.567616,0.450986,0.411073,0.477259,0.590838,0.432185,0.441896,0.44768,0.458179,...,0.545943,0.596377,0.64331,0.411073,0.330781,0.673923,0.808258,0.921122,0.362918,0.363197
2357,NaN,0.590661,0.483118,0.446346,0.517119,0.633998,0.462151,0.477644,0.48995,0.505028,...,0.570604,0.622106,0.665234,0.446346,0.336697,0.707881,0.81962,0.742729,0.921122,0.362918


In [168]:
hands

,Unnamed: 0,video,letter,handedness,x0,x1,x2,x3,x4,x5,...,y18,y19,y20,bbox_x1,bbox_y1,bbox_x2,bbox_y2,iou0,iou1,iou2
0,0,vid3,A,Right,0.362896,0.418465,0.486683,0.546821,0.595302,0.463339,...,0.321185,0.344583,0.349709,0.320133,0.297793,0.595302,0.437581,NaN,NaN,NaN
1,1,vid3,A,Right,0.358468,0.413572,0.485543,0.546373,0.594428,0.468047,...,0.319683,0.342699,0.347909,0.322879,0.297573,0.594428,0.433684,0.957844,NaN,NaN
2,2,vid3,A,Right,0.360684,0.413036,0.483942,0.544283,0.593048,0.463905,...,0.319794,0.342711,0.347140,0.322172,0.298325,0.593048,0.435321,0.975200,0.957844,NaN
3,3,vid3,A,Right,0.349367,0.405749,0.475828,0.534505,0.581794,0.451813,...,0.314091,0.339722,0.347585,0.303982,0.288935,0.581794,0.428068,0.800520,0.975200,0.957844
4,4,vid3,A,Right,0.354493,0.408133,0.479159,0.538913,0.586343,0.455486,...,0.322346,0.346321,0.351305,0.311088,0.297836,0.586343,0.437049,0.845080,0.800520,0.975200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2354,2354,vid4,YA,Left,0.664645,0.587001,0.568673,0.646658,0.749332,0.519975,...,0.570490,0.614499,0.652892,0.519975,0.325261,0.799866,0.833102,0.363197,0.782826,0.852497
2355,2355,vid4,YA,Left,0.560196,0.458429,0.417879,0.485016,0.596180,0.428843,...,0.565826,0.610831,0.650451,0.417879,0.346134,0.669148,0.806221,0.362918,0.363197,0.782826
2356,2356,vid4,YA,Left,0.567616,0.450986,0.411073,0.477259,0.590838,0.432185,...,0.545943,0.596377,0.643310,0.411073,0.330781,0.673923,0.808258,0.921122,0.362918,0.363197
2357,2357,vid4,YA,Left,0.590661,0.483118,0.446346,0.517119,0.633998,0.462151,...,0.570604,0.622106,0.665234,0.446346,0.336697,0.707881,0.819620,0.742729,0.921122,0.362918


In [164]:
mms.transform([hands.loc[23, xs_cols]])

/home/avgustina/.local/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


array([[0.62184223, 0.4749928 , 0.42535142, 0.42325838, 0.43577345,
        0.54588163, 0.57996182, 0.5288644 , 0.53171355, 0.60992123,
        0.60577991, 0.5577898 , 0.54877282, 0.58950953, 0.63635273,
        0.60227079, 0.56393713, 0.59359934, 0.65707638, 0.66733015,
        0.64745944]])

In [114]:
mms = MinMaxScaler()
scaled_xs = mms.fit_transform(hands[xs_cols])
scaled_xs

array([[0.34327555, 0.36074164, 0.42762489, ..., 0.31239565, 0.33412133,
        0.34251351],
       [0.33781847, 0.35530641, 0.4265002 , ..., 0.3150316 , 0.33579492,
        0.34459643],
       [0.34054969, 0.35471114, 0.4249213 , ..., 0.31435348, 0.33525976,
        0.34376465],
       ...,
       [0.59558427, 0.3968666 , 0.35302594, ..., 0.64337668, 0.61703666,
        0.58805062],
       [0.62398683, 0.43256097, 0.38782764, ..., 0.67214304, 0.64748762,
        0.6204724 ],
       [0.6152225 , 0.42434845, 0.38223289, ..., 0.67654297, 0.64964855,
        0.61755549]])

In [130]:
sxdf = pd.DataFrame(scaled_xs)
pd.DataFrame(hands.handedness)

,handedness
0,Right
1,Right
2,Right
3,Right
4,Right
...,...
2354,Left
2355,Left
2356,Left
2357,Left


In [135]:
xs_handedness = pd.concat([sxdf, pd.DataFrame(hands.handedness)], axis = 1)

In [137]:
xs_handedness

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,handedness
0,0.343276,0.360742,0.427625,0.497612,0.565060,0.473670,0.450075,0.410488,0.454478,0.430314,...,0.406632,0.329672,0.335349,0.334593,0.341739,0.294053,0.312396,0.334121,0.342514,Right
1,0.337818,0.355306,0.426500,0.497214,0.564355,0.478339,0.453984,0.411870,0.455474,0.434350,...,0.406251,0.332090,0.336374,0.333724,0.340885,0.296655,0.315032,0.335795,0.344596,Right
2,0.340550,0.354711,0.424921,0.495360,0.563244,0.474231,0.452701,0.410758,0.454290,0.430980,...,0.405711,0.329699,0.335727,0.333588,0.340666,0.294825,0.314353,0.335260,0.343765,Right
3,0.326601,0.346616,0.416915,0.486681,0.554174,0.462239,0.431697,0.394137,0.443458,0.416500,...,0.392933,0.315945,0.317217,0.317169,0.327396,0.282261,0.296888,0.318131,0.328925,Right
4,0.332919,0.349264,0.420202,0.490593,0.557840,0.465882,0.440090,0.399971,0.445252,0.420977,...,0.397324,0.319815,0.324544,0.324029,0.331483,0.285529,0.303710,0.324525,0.333424,Right
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2354,0.715170,0.547959,0.508519,0.586220,0.689191,0.529835,0.551696,0.538008,0.573941,0.639190,...,0.580724,0.662176,0.729416,0.686562,0.633853,0.699595,0.773019,0.766892,0.726424,Left
2355,0.586440,0.405135,0.359742,0.442758,0.565767,0.439460,0.437424,0.418294,0.464475,0.535685,...,0.456175,0.561587,0.595126,0.540460,0.497346,0.605049,0.645669,0.624876,0.592931,Left
2356,0.595584,0.396867,0.353026,0.435873,0.561463,0.442774,0.447715,0.421231,0.462686,0.537300,...,0.460180,0.561334,0.585156,0.532549,0.496614,0.609515,0.643377,0.617037,0.588051,Left
2357,0.623987,0.432561,0.387828,0.471251,0.596245,0.472491,0.483464,0.460397,0.500066,0.573461,...,0.496286,0.597357,0.618142,0.565849,0.529576,0.641274,0.672143,0.647488,0.620472,Left


In [136]:
ctr = FunctionTransformer(custom_transformer_np)
ctr.fit_transform(xs_handedness)

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

In [54]:
test.loc[test['handedness'] == 'Left']

,handedness,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
23,Left,0.588921,0.521316,0.484378,0.463045,0.434875,0.536156,0.574139,0.563843,0.544691,...,0.596671,0.576882,0.623940,0.651897,0.634186,0.611518,0.656905,0.679113,0.665428,0.647037
24,Left,0.495599,0.408037,0.358230,0.333148,0.307225,0.442930,0.439005,0.436306,0.434064,...,0.479657,0.477527,0.537376,0.540522,0.523695,0.521221,0.583774,0.591823,0.569848,0.560928
25,Left,0.511759,0.400714,0.340093,0.304574,0.260546,0.425462,0.413033,0.421482,0.427223,...,0.478417,0.478958,0.572343,0.562320,0.541198,0.539648,0.634493,0.639421,0.607527,0.597441
26,Left,0.492689,0.406212,0.357284,0.330062,0.304960,0.439958,0.434181,0.431993,0.430875,...,0.476228,0.474904,0.532671,0.535859,0.518601,0.516686,0.578470,0.587302,0.566045,0.556634
27,Left,0.509198,0.447768,0.421085,0.423309,0.422584,0.500459,0.498768,0.482038,0.482419,...,0.514362,0.519324,0.586356,0.589634,0.556561,0.553472,0.627577,0.633677,0.599377,0.589605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2354,Left,0.664645,0.587001,0.568673,0.646658,0.749332,0.519975,0.545874,0.573711,0.597614,...,0.613049,0.613037,0.693922,0.744313,0.720626,0.683957,0.770240,0.799866,0.766680,0.726671
2355,Left,0.560196,0.458429,0.417879,0.485016,0.596180,0.428843,0.431605,0.444510,0.460421,...,0.485995,0.472104,0.597049,0.610956,0.570801,0.542523,0.669148,0.667233,0.622253,0.592047
2356,Left,0.567616,0.450986,0.411073,0.477259,0.590838,0.432185,0.441896,0.447680,0.458179,...,0.490762,0.476636,0.596805,0.601056,0.562688,0.541764,0.673923,0.664845,0.614281,0.587125
2357,Left,0.590661,0.483118,0.446346,0.517119,0.633998,0.462151,0.477644,0.489950,0.505028,...,0.526152,0.517491,0.631497,0.633813,0.596836,0.575916,0.707881,0.694805,0.645249,0.619822


In [93]:
test = pd.DataFrame(hands, columns = ['handedness'] + xs_cols)
ttest = test.copy()
ttest.loc[ttest.handedness == 'Left', xs_cols] = ttest.loc[ttest.handedness == 'Left', xs_cols].apply(lambda x: x - 1)

In [106]:
test.loc[23]

handedness        Left
x0            0.588921
x1            0.521316
x2            0.484378
x3            0.463045
x4            0.434875
x5            0.536156
x6            0.574139
x7            0.563843
x8            0.544691
x9            0.582305
x10           0.612463
x11           0.596671
x12           0.576882
x13            0.62394
x14           0.651897
x15           0.634186
x16           0.611518
x17           0.656905
x18           0.679113
x19           0.665428
x20           0.647037
Name: 23, dtype: object

In [107]:
ttest.loc[23]

handedness        Left
x0           -0.411079
x1           -0.478684
x2           -0.515622
x3           -0.536955
x4           -0.565125
x5           -0.463844
x6           -0.425861
x7           -0.436157
x8           -0.455309
x9           -0.417695
x10          -0.387537
x11          -0.403329
x12          -0.423118
x13           -0.37606
x14          -0.348103
x15          -0.365814
x16          -0.388482
x17          -0.343095
x18          -0.320887
x19          -0.334572
x20          -0.352963
Name: 23, dtype: object

In [105]:
ttest.loc[ttest.handedness == 'Left', xs_cols]

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20
23,-0.411079,-0.478684,-0.515622,-0.536955,-0.565125,-0.463844,-0.425861,-0.436157,-0.455309,-0.417695,...,-0.403329,-0.423118,-0.376060,-0.348103,-0.365814,-0.388482,-0.343095,-0.320887,-0.334572,-0.352963
24,-0.504401,-0.591963,-0.641770,-0.666852,-0.692775,-0.557070,-0.560995,-0.563694,-0.565936,-0.509509,...,-0.520343,-0.522473,-0.462624,-0.459478,-0.476305,-0.478779,-0.416226,-0.408177,-0.430152,-0.439072
25,-0.488241,-0.599286,-0.659907,-0.695426,-0.739454,-0.574538,-0.586967,-0.578518,-0.572777,-0.498420,...,-0.521583,-0.521042,-0.427657,-0.437680,-0.458802,-0.460352,-0.365507,-0.360579,-0.392473,-0.402559
26,-0.507311,-0.593788,-0.642716,-0.669938,-0.695040,-0.560042,-0.565819,-0.568007,-0.569125,-0.514241,...,-0.523772,-0.525096,-0.467329,-0.464141,-0.481399,-0.483314,-0.421530,-0.412698,-0.433955,-0.443366
27,-0.490802,-0.552232,-0.578915,-0.576691,-0.577416,-0.499541,-0.501232,-0.517962,-0.517581,-0.457137,...,-0.485638,-0.480676,-0.413644,-0.410366,-0.443439,-0.446528,-0.372423,-0.366323,-0.400623,-0.410395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2354,-0.335355,-0.412999,-0.431327,-0.353342,-0.250668,-0.480025,-0.454126,-0.426289,-0.402386,-0.390798,...,-0.386951,-0.386963,-0.306078,-0.255687,-0.279374,-0.316043,-0.229760,-0.200134,-0.233320,-0.273329
2355,-0.439804,-0.541571,-0.582121,-0.514984,-0.403820,-0.571157,-0.568395,-0.555490,-0.539579,-0.485914,...,-0.514005,-0.527896,-0.402951,-0.389044,-0.429199,-0.457477,-0.330852,-0.332767,-0.377747,-0.407953
2356,-0.432384,-0.549014,-0.588927,-0.522741,-0.409162,-0.567815,-0.558104,-0.552320,-0.541821,-0.484430,...,-0.509238,-0.523364,-0.403195,-0.398944,-0.437312,-0.458236,-0.326077,-0.335155,-0.385719,-0.412875
2357,-0.409339,-0.516882,-0.553654,-0.482881,-0.366002,-0.537849,-0.522356,-0.510050,-0.494972,-0.451200,...,-0.473848,-0.482509,-0.368503,-0.366187,-0.403164,-0.424084,-0.292119,-0.305195,-0.354751,-0.380178
